In [2]:
#libraries for classfication
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression #Regression
from sklearn.linear_model import LinearRegression #Regression
from sklearn.metrics import mean_squared_error, r2_score #Regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,PrecisionRecallDisplay
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
#for precision recall curve
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import label_binarize
from collections import Counter

PREVISUALIZACION DE LOS DATOS

In [3]:
def show_correlation_matrix(df):
    corr = df.corr()
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr, annot=True, fmt=".2f", square=True, cbar=True)
    plt.show()

In [4]:
#DATASET LOADING
#ESTE ES EL DATASET SIN MODIFICAR
df = pd.read_csv("B.HEALTH_classification.csv")

In [ ]:
#GRAFICA DEL DATASET
#GRAFICAMOS CANTIDAD DE DATOS POR CLASE
#0 es buena calidad 2 es mala calidad
sns.countplot(x='label', data=df)
plt.show()
print(df["label"].value_counts())

In [ ]:
#MOSTRAR MATRIZ DE CORRELACION
base_df = df.drop(columns=["subject","day"])
columns = [] #si queremos ver la matriz de correlacion sin alguna columna agregarlo aqui
show_correlation_matrix(base_df.drop(columns=columns))

 PREPARACION DE LOS DATOS PARA EL ENTRENAMIENTO Y VISUALIZACION DE CARACTERISTICAS

In [7]:
def normalize_data(X, how="minmax"):
    if how == "minmax":
        scaler = MinMaxScaler()
    elif how == "standard":
        scaler = StandardScaler()
    return scaler.fit_transform(X)

In [8]:
def apply_tsne(X, n_components=2, pca=False):
    if pca:
        reduccion = TSNE(n_components, init='pca')
    else:
        reduccion = TSNE(n_components)
    x_new = reduccion.fit_transform(X)
    return x_new

In [9]:
def plot_data_TSNE(x,y): #plot with TSNE and PCA to reduce dimensions
    fig, ax = plt.subplots(1, 1, figsize=(12, 5))
    
    x_new = apply_tsne(x, n_components=2, pca=True)
    tmp_df = pd.DataFrame(np.column_stack([x_new, y]))
    tmp_df.columns = ["x1", "x2", "Y"]
    
    sns.scatterplot(x="x1", y="x2", hue="Y", data=tmp_df, ax=ax)
    

In [10]:
#separate test and train data
def split_data(X, y, test_size=0.3):
    return train_test_split(X, y, test_size=test_size, random_state=42)

In [11]:
#REMOVEMOS COLUMNAS QUE NO QUEREMOS Y APLICAMOS SMOTE DE SER NECESARIO
remove_columns = ["subject","day", "label"] #remover columnas que no queremos para las variables
use_smote = True

X, Y = df.drop(columns=remove_columns), df["label"]

if use_smote:
    smote = SMOTE(sampling_strategy='minority')
    X, Y = smote.fit_resample(df.drop(columns=remove_columns), df["label"])
#separamos en X variables y Y labels



In [12]:
#NORMALIZAMOS LOS DATOS
X = normalize_data(X, how="minmax")

In [13]:
#SEPARAMOS LOS DATOS DE ENTRENAMIENTO Y PRUEBA
X_train, X_test, y_train, y_test = split_data(X, Y)


In [ ]:
#PLOTTING DATA
plot_data_TSNE(X, Y)

ENTRENAMIENTO Y TUNNING DE MODELOS PARA CLASIFICACION

In [15]:
def select_model(model_name="DecisionTree"): 
    if model_name == "DecisionTree":
        return DecisionTreeClassifier()
    elif model_name == "SVM":
        return SVC()
    elif model_name == "RandomForest":
        return RandomForestClassifier()
    elif model_name == "XGBoost":
        return XGBClassifier()
    elif model_name == "MLP":
        return MLPClassifier()
    else:
        print("Invalid model name")
        return None

In [16]:
#comparing each model accuracy
def compare_models_accuracy(X_train, y_train, X_test, y_test):
    models = ["DecisionTree", "SVM", "RandomForest"]
    accuracies = []
    for model_name in models:
        model = select_model(model_name)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    return list(zip(models, accuracies))
        

In [17]:
def compare_models_accuracy(X_train, y_train, X_test, y_test):
    models = ["DecisionTree", "SVM", "RandomForest", "XGBoost"]
    accuracies = []
    for model_name in models:
        model = select_model(model_name)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    return list(zip(models, accuracies))

In [ ]:
#plotting comparison of models with no hyperparameter tuning
comparison_list = compare_models_accuracy(X_train, y_train, X_test, y_test)
print(comparison_list)


In [19]:
def parameter_tunning(model,parameters, method="grid", scoring_metric = "accuracy"):
    if method == "grid":
        clf = GridSearchCV(model, param_grid=parameters, scoring=scoring_metric, cv=5, n_jobs=-1)  
    # clf_svc = RandomizedSearchCV(clf_svc, param_distributions=parameters, scoring="accuracy", cv=5, n_jobs=-1, n_iter=15)
    else:
        clf = RandomizedSearchCV(model, param_distributions=parameters, scoring=scoring_metric, cv=5, n_jobs=-1, n_iter=15)
    
    clf.fit(X_train, y_train)
    
    print("-----------------")
    print("Model", model.__class__.__name__)
    print("Best score : ", clf.best_score_)
    print("Best Parameters : ", clf.best_params_)
    print("-----------------")
    return clf.cv_results_

In [20]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    confusion = confusion_matrix(y_test, y_pred)
    print("Model", model.__class__.__name__)
    print("Accuracy: ", accuracy)
    print("F1 Score: ", f1)
    print("Confusion Matrix: ")
    print(confusion)
    return accuracy, f1, confusion

In [21]:
# parameters = {
#    "n_estimators":[i for i in range(1,200)],                   
#    "learning_rate":[0.1*i for i in range(1,10)],
#    "max_depth":[i for i in range(1,20)],                                            
# }
# useGridSearch = False
# parameter_tunning(select_model("XGBoost"), parameters, useGridSearch)

In [22]:
#DecisionTree parameters
weights = np.linspace(0.0,0.99,20)
# parameters = {
#     "criterion": ["gini", "entropy","log_loss"],
#     "max_depth": [i if i!=0 else None for i in range(1, 10)],
#     "max_features": ["auto", "sqrt", "log2"],
#     'class_weight': [{0:x, 1:1.0-x , 2:1.0 - x} for x in weights]
# }
# model = DecisionTreeClassifier()
# parameter_tunning(model, parameters, scoring_metric="f1_weighted")

In [23]:
# decision_tree = DecisionTreeClassifier(criterion="entropy", max_depth=9, max_features="sqrt")
# decision_tree.fit(X_train, y_train)
# y_pred = decision_tree.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy DecisionTree: ", accuracy)

In [ ]:
svm = SVC()
#weights = np.linspace(0.0,0.99,200)
parameters = {
    "C": [0.1, 1, 10, 100],
    "kernel": ["linear", "poly", "rbf", "sigmoid"], 
    "gamma": [0.01*i for i in range(1, 100)],
    #'class_weight': [{0:x, 1:1.0-x,2:1.0 - x} for x in weights]
}
parameter_tunning(svm, parameters,scoring_metric="f1_weighted")

In [ ]:
svm = SVC(C=10, degree=2, gamma=0.44, kernel="rbf")
clf = svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy SVM: ", accuracy)

In [26]:

def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in"""
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

In [27]:
def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier."""
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

In [ ]:
#APPLYTING TSNE TO VISUALIZE DATA WITH SVC
plot_data_TSNE(X, Y)
plt.show()
X_tsne = apply_tsne(X, n_components=2, pca=True)
X_train_tsne, X_test_tsne, y_train_tsne, y_test_tsne = split_data(X_tsne, Y)
clf = svm.fit(X_train_tsne, y_train_tsne)

 # Create a meshgrid for plotting
X0, X1 = X_tsne[:, 0], X_tsne[:, 1]
xx, yy = make_meshgrid(X0, X1)

# Plot the decision boundaries
fig, ax = plt.subplots()
plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax.scatter(X0, X1, c=Y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
ax.set_xlabel('TSNE Component 1')
ax.set_ylabel('TSNE Component 2')
ax.set_title('SVM Decision Boundary')
plt.show()



In [29]:
# random_f= RandomForestClassifier()

# parameters = {
#     "n_estimators": [50, 100, 200, 300],
#     "criterion": ["gini", "entropy","log_loss"],
#     "max_depth": [i if i!=0 else None for i in range(1, 10)],
#     "max_features": ["auto", "sqrt", "log2"],
#     'class_weight': [{0:x, 1:1.0-x , 2:1.0 - x} for x in weights]
# }
# parameter_tunning(random_f, parameters)

In [30]:
# random_f= RandomForestClassifier(criterion="entropy", max_depth=5, max_features="log2", n_estimators=50)
# random_f.fit(X_train, y_train)
# y_pred = random_f.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy RandomForest: ", accuracy)

In [31]:
# xgboost = XGBClassifier()
# parameters = {
#    "n_estimators":[i for i in range(1,200)],                   
#    "learning_rate":[0.1*i for i in range(1,10)],
#    "max_depth":[i for i in range(1,20)],
#    "max_features":[i for i in range(1,20)],
#    'class_weight': [{0:x, 1:1.0-x , 2:1.0 - x} for x in weights]
                                                  
# }
# parameter_tunning(xgboost, parameters, method="random", scoring_metric="f1_weighted")

In [32]:
# mlp_model = MLPClassifier()
# parameters = {
#     "hidden_layer_sizes": [(100,), (200,), (300,)],
#     "activation": ["identity", "logistic", "tanh", "relu"],
#     "solver": ["lbfgs", "sgd", "adam"],
#     "alpha": [0.0001, 0.001, 0.01, 0.1],
#     "learning_rate": ["constant", "invscaling", "adaptive"]
# }
# parameter_tunning(mlp_model, parameters)

In [33]:
# mlp_model = MLPClassifier(activation="relu", alpha=0.1, hidden_layer_sizes=(100,), learning_rate="invscaling", solver="adam")
# mlp_model.fit(X_train, y_train)
# y_pred = mlp_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy MLP: ", accuracy)

In [ ]:
def decimal_to_time(decimal_time): ##calculo para pasar tiempo decimal a horas minutos y segundos
    """
    Convert a decimal time representation to HH:MM:SS format.
    
    Parameters:
    decimal_time (float): The decimal representation of time.
    
    Returns:
    str: The time in HH:MM:SS format.
    """
    total_seconds = int(decimal_time * 24 * 3600)
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{hours:02}:{minutes:02}:{seconds:02}"


print(decimal_to_time(0.457638889)) 
print(decimal_to_time(0.707638889)) 

ANALISIS Y COMPARACION DE MODELOS


In [35]:
def graph_precision_recall_curve(y_test, y_score, n_classes=3):
    precision = dict()
    recall = dict()
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],
                                                            y_score[:, i])
        plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))
        
    plt.xlabel("recall")
    plt.ylabel("precision")
    plt.legend(loc="best")
    plt.title("precision vs. recall curve")
    plt.show()

In [ ]:
#classifiers with best hyperparameters
#SMOTE
decision_tree = DecisionTreeClassifier(criterion="entropy", max_depth=5, max_features="log2")
xgbModel = XGBClassifier(learning_rate=0.2, max_depth=19, n_estimators=154)
svm = SVC(C=100, degree=2, gamma=0.09, kernel="rbf")
random_f= RandomForestClassifier(criterion="entropy", max_depth=6, max_features="sqrt", n_estimators=50)
mlp_model = MLPClassifier(activation="relu", alpha=0.01, hidden_layer_sizes=(200,), learning_rate="invscaling", solver="adam")

#WEIGHTED
# decision_tree = DecisionTreeClassifier(criterion="log_loss", max_depth=6, max_features="sqrt",class_weight={0: np.float64(0.5074371859296483), 1: np.float64(0.49256281407035174), 2: np.float64(0.49256281407035174)})
# xgbModel = XGBClassifier(learning_rate=0.4, max_depth=15, n_estimators=10,class_weight={0: np.float64(0.8336842105263158), 1: np.float64(0.1663157894736842), 2: np.float64(0.1663157894736842)})
# svm = SVC(C=100, gamma=0.03, kernel="rbf",probability=True,class_weight={0: 0.5223618090452261, 1:0.4776381909547739, 2:0.4776381909547739}) 
# random_f= RandomForestClassifier(criterion="log_loss", max_depth=9, max_features="log2", n_estimators=50 , class_weight= {0: np.float64(0.10421052631578948), 1: np.float64(0.8957894736842105), 2: np.float64(0.8957894736842105)} )
# mlp_model = MLPClassifier(activation="relu", alpha=0.01, hidden_layer_sizes=(200,), learning_rate="invscaling", solver="adam")
train_and_evaluate_model(decision_tree, X_train, y_train, X_test, y_test)
train_and_evaluate_model(svm, X_train, y_train, X_test, y_test)
train_and_evaluate_model(random_f, X_train, y_train, X_test, y_test)
train_and_evaluate_model(xgbModel, X_train, y_train, X_test, y_test)
train_and_evaluate_model(mlp_model, X_train, y_train, X_test, y_test)

In [ ]:
#WEIGHING CLASSES
from sklearn.model_selection import GridSearchCV, StratifiedKFold
svm = SVC(probability=True)

weights = np.linspace(0.0,0.99,200)
param_grid = {'class_weight': [{0:x, 1:1.0-x} for x in weights]}
#Fitting grid search to the train data with 5 folds
gridsearch_csv_results = parameter_tunning(svm, param_grid, method="grid")

#Ploting the score for different values of weight
sns.set_style('whitegrid')
plt.figure(figsize=(12,8))
weigh_data = pd.DataFrame({ 'score': gridsearch_csv_results['mean_test_score'], 'weight': (1- weights)})
sns.lineplot(data=weigh_data, y='score', x='weight')
plt.xlabel('Weight for class 1')
plt.ylabel('F1 score')
plt.xticks([round(i/10,1) for i in range(0,11,1)])
plt.title('Scoring for different class weights', fontsize=24)

In [ ]:
#Testing model with best hyperparameters and weights or smote
#model = SVC(C=100, gamma=0.03, kernel="rbf",probability=True,class_weight={0: 0.5223618090452261, 1:0.4776381909547739, 2:0.4776381909547739}) #model without smote and applying weights
model = SVC(C=100, degree=2, gamma=0.09, kernel="rbf",probability=True) #model with smote
train_and_evaluate_model(model, X_train, y_train, X_test, y_test)

n_classes = 3
Y_pr = label_binarize(Y, classes=[0, 1, 2])#esto es necesario para la curva de precision recall
X_train_pr, X_test_pr, y_train_pr, y_test_pr = train_test_split(X,
                                                    Y_pr,
                                                    random_state = 42)
clf = OneVsRestClassifier(model)
clf.fit(X_train_pr, y_train_pr)
y_score = clf.predict_proba(X_test_pr)

graph_precision_recall_curve(y_test_pr, y_score, n_classes)



REGRESION

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression

# Preparar los datos
new_df_regression = df.drop(['subject','label'], axis=1)
new_X = new_df_regression.drop('calories burnt (kcal)', axis=1)
new_Y = new_df_regression['calories burnt (kcal)']

# Filas con valores nulos
new_X = new_X.dropna()
new_Y = new_Y[new_X.index]

# Calcular la correlación
correlation_matrix = new_df_regression.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Matriz de Correlación')
plt.show()

# Seleccionar las variables más correlacionadas con la variable objetivo
correlation_threshold = 0.5
correlated_features = correlation_matrix.index[abs(correlation_matrix['calories burnt (kcal)']) > correlation_threshold].tolist()
correlated_features.remove('calories burnt (kcal)')
new_X = new_X[correlated_features]

# Separar en datos de entrenamiento y prueba
# new_X_train, new_X_test, new_Y_train, new_Y_test = split_data(new_X, new_Y)

# # Normalizamos los datos
# new_X_train = normalize_data(new_X_train, how="minmax")
# new_X_test = normalize_data(new_X_test, how="minmax")

# Normalizar los datos
scaler = StandardScaler()
new_X = scaler.fit_transform(new_X)

# Entrenar el modelo
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(max_depth=5),
    "Random Forest": RandomForestRegressor(n_estimators=50, max_depth=5),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3),
    "Support Vector Regression": SVR(C=100, gamma=0.01)
}

# Evaluar cada modelo con validación cruzada
for name, model in models.items():
    scores = cross_val_score(model, new_X, new_Y, cv=5, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    mae = -cross_val_score(model, new_X, new_Y, cv=5, scoring='neg_mean_absolute_error').mean()
    r2 = cross_val_score(model, new_X, new_Y, cv=5, scoring='r2').mean()
    rmse = np.sqrt(mse)
    
    print(f"{name}:")
    print(f"  Mean Squared Error: {mse}")
    print(f"  Mean Absolute Error: {mae}")
    print(f"  R² Score: {r2}")
    print(f"  Root Mean Squared Error: {rmse}")
    print()
    
    # Ajustar el modelo y predecir
    model.fit(new_X, new_Y)
    new_Y_pred = model.predict(new_X)
    
    # Graficar resultados
    plt.figure(figsize=(12, 6))
    
    # Gráfico de dispersión de valores reales vs. predichos
    plt.subplot(1, 2, 1)
    plt.scatter(new_Y, new_Y_pred, alpha=0.5)
    plt.plot([new_Y.min(), new_Y.max()], [new_Y.min(), new_Y.max()], 'k--', lw=2)
    plt.xlabel('Valores Reales')
    plt.ylabel('Valores Predichos')
    plt.title(f'{name} - Valores Reales vs. Predichos')
    
    # Gráfico de residuos
    plt.subplot(1, 2, 2)
    residuals = new_Y - new_Y_pred
    plt.scatter(new_Y_pred, residuals, alpha=0.5)
    plt.hlines(y=0, xmin=new_Y_pred.min(), xmax=new_Y_pred.max(), colors='r', linestyles='dashed')
    plt.xlabel('Valores Predichos')
    plt.ylabel('Residuos')
    plt.title(f'{name} - Gráfico de Residuos')
    
    plt.tight_layout()
    plt.show()